In [1]:
import pandas as pd
import nltk
import re

In [2]:
data = pd.read_csv('depression.csv', lineterminator='\n')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957593 entries, 0 to 957592
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   title      957589 non-null  object
 1   self_text  851537 non-null  object
dtypes: object(2)
memory usage: 14.6+ MB


In [4]:
data['title'] = data['title'].astype('str')
data['self_text'] = data['self_text'].astype('str')

In [5]:
data.head()

,title,self_text
0,So many kinds of therapists...what to choose?,Pretty simply question here. I've been thinkin...
1,i'm drawing a comic thing,---
2,Tired of rejection..,I'm 20 [M] and never had a real girlfriend. Th...
3,Everyone thinks I'm fine.. but I'm no where close,I don't really mind if anyone reads or cares a...
4,honesty,I drink most nights because I am unhappy. I a...


In [6]:
data['texts'] = data['title']+" "+data['self_text']

In [7]:
data.drop(labels=['title', 'self_text'], axis=1, inplace = True)

In [8]:
data.head()

,texts
0,So many kinds of therapists...what to choose? ...
1,i'm drawing a comic thing ---
2,Tired of rejection.. I'm 20 [M] and never had ...
3,Everyone thinks I'm fine.. but I'm no where cl...
4,honesty I drink most nights because I am unhap...


In [9]:
words_of_interest = ['depressed', 'depression', 'stress', 'help', 
                     'suicide', 'devastated', 'frustrated',
                    "upsetting", "distressing", "painful", "heartbreaking", 
                     "dispiriting", "disheartening", "discouraging", "demoralizing"]

In [10]:
def search_word(data, words):
    for each_word in words:
        if re.search(each_word, data):
            return 1
    return 0

In [11]:
data['count'] = data['texts'].apply(lambda x: search_word(x, words_of_interest))


In [12]:
tot = data['count'].sum()
print(f'Number of posts that contain depressed words: {tot}')

Number of posts that contain depressed words: 514868


## Tokenize and Clean

In [16]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/student.unimelb.edu.au/jprasetiyo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
data['tokenized_text'] = data['texts']

In [17]:
data['tokenized_text'] = data["tokenized_text"].apply(lambda x: x.lower()).apply(word_tokenize)

In [18]:
stop_words = [ 'what', 'which', 'who', 'whom', 'this', 'that', "that'll",'these', 'those', 'am', 'is', 'was', 'were',
 'be','been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but',
 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
 'through', 'during', 'before','after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on','off', 'over',
 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each',
 'few', 'more', 'most', 'other', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can',
 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't",
 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn',
  "isn't", 'ma', 'mightn',"mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn',
 "wasn't", 'weren', "weren't", 'won',"won't", 'wouldn',"wouldn't", "http"]


# remove stop words
data['tokenized_text'] = data["tokenized_text"].apply(lambda x: [word for word in x if word not in stop_words])

In [20]:
# remove non alphanumeric words
data['tokenized_text'] = data["tokenized_text"].apply(lambda x: [word for word in x if word.isalnum()])

In [21]:
data.to_csv("depression_tokenized.csv", index=False)

In [22]:
data

,texts,count,tokenized_text
0,So many kinds of therapists...what to choose? ...,1,"[many, kinds, therapists, choose, pretty, simp..."
1,i'm drawing a comic thing ---,0,"[i, drawing, comic, thing]"
2,Tired of rejection.. I'm 20 [M] and never had ...,0,"[tired, i, 20, never, real, girlfriend, relati..."
3,Everyone thinks I'm fine.. but I'm no where cl...,1,"[everyone, thinks, i, i, close, i, really, min..."
4,honesty I drink most nights because I am unhap...,0,"[honesty, i, drink, nights, i, unhappy, i, unh..."
...,...,...,...
957588,Someone please answer As you can see by the ti...,1,"[someone, please, answer, you, see, title, my,..."
957589,I've usually been the person to show up and he...,1,"[i, usually, person, show, help, everyone, i, ..."
957590,My life is just kinda shit I'm a college stude...,0,"[my, life, kinda, shit, i, college, student, i..."
957591,I survived A few months ago I survived an over...,0,"[i, survived, months, ago, i, survived, overdo..."


In [23]:
from nltk.stem import PorterStemmer
ps = PorterStemmer() 

data['tokenized_text'] = data["tokenized_text"].apply(lambda x: [ps.stem(word) for word in x])

In [25]:
data.to_csv("depression_tokenised_stemmed.csv", index = False)